![Top <](./images/watsonxdata.png "watsonxdata")

# Ollama and Open WebUI

Ollama is an open-source project that serves as a powerful and user-friendly platform for running LLMs (e.g., IBM Instructlab, Llama 3, Mistral, Gemma 2) on your local machine.  More information on Ollama is available at [https://ollama.com/](https://ollama.com). 

Ollama is accessed via Open WebUI, which is a self-hosted interface for AI. Details on Open WebUI can be found at [https://openwebui.com/](https://openwebui.com/). 

This notebook installs these services in your environment to demonstrate how AI models can be enhanced using prompts generated by Milvus. The prompt used in this notebook was generated by the Milvus Example notebook. Explore the Milvus Example notebook to learn how prompts are generated.

**Note**: This system is provided for you to prototype the use of RAG enhanced prompts into an AI model. Production systems would use watsonx.ai for managing and deploying models. Since this is a small environment, the response time to answer questions will be very long.

## Credentials and Server Addresses
The Open WebUI console requires the name or address of the server that will be running the console, along with an open port on your server. This value will be different for each user of this system. Gather the following information from your server reservation:

* Watsonx Server URL - This will be the URL at the beginning of all service addresses. For example: `useast.services.cloud.techzone.ibm.com`.
* Open WebUI External Port - Use Open port #1 or #2 in the reservation, depending on which one is free.
* Open WebUI Internal Port - Use 10000 for open port #1 or 10001 for open port #2

Fill in the values in the next cell and execute it.

In [ ]:
watsonx_server_url   = "useast.services.cloud.techzone.ibm.com"
WebUI_external_port  = "12345"
WebUI_internal_port  = "10000"
WebUI_URL           = f"http://{watsonx_server_url}:{WebUI_external_port}"

### Start the Open WebUI and Ollama Service
The next command will start a docker which contains the Open WebUI and the Ollama Service.

**Note:** If you are restarting this service, and want to keep the models that you have previously loaded, make sure to comment out the line in the script below which deletes the two persistent volumes used by the Ollama service.

In [ ]:
%system docker stop open-webui
%system docker rm open-webui
# --> Comment out the next line IF you want to keep the models you have loaded
%system docker volume rm -f open-webui ollama
%system docker run -d -p {WebUI_internal_port}:8080 \
  -v ollama:/root/.ollama -v open-webui:/app/backend/data \
  --name open-webui \
  --restart always \
  --env WEBUI_AUTH=false \
  ghcr.io/open-webui/open-webui:ollama

Check that the service is running.

In [ ]:
%system docker ps | grep open-webui

## Add a Model to Ollama
Before we start using the Open WebUI interface, we must load a model into Ollama. In order to load a model, we must use the Ollama CLI command. The models that can be loaded can be found in a variety of locations, but for this notebook, we are going to use the Hugging Face website at [https://huggingface.co/](https://huggingface.co/).

![Higging Face](images/ollama-huggingface.png)

We are going to select the Instructlab Granite-7b-lab. To view the assets available in Instructlab, type Instructlab in the HuggingFace search box and hit enter. 
> InstructLab (based on the Large-scale Alignment for ChatBots technique) is an innovative open-source initiative led by Red Hat and IBM. The project aims to enhance the capabilities of Large Language Models (LLMs) through a community-driven approach that leverages a novel taxonomy-based curation process and synthetic data generation. InstructLab provides tools for users to engage with and improve LLMs, contributing skills and knowledge to the project’s taxonomy repository.

![Instructlab](images/ollama-instructlab.png)

Scroll to the bottom of the page and you will see the various models that are available.

![Models](images/ollama-instructlab-models.png)

### Load Model into Ollama
The next command will load a model into Ollama. You can add as many models as you wish, but be aware that models may take a long time to download and they take up considerable amount of space. The `granite-7b-lab` will take about 5 minutes to load into the system.

In [ ]:
model = "instructlab/granite-7b-lab"
%system docker exec open-webui ollama pull {model}

## Connect to the Console
The next command will print the URL for the Attu console. Click on the link that is generated.

In [ ]:
WebUI_URL

The Open WebUI console should display. You may see a brief message regarding authentication. There is no need to log onto the system.
![OpenWebUI](images/ollama-open-webui.png)

The `granite-7b-lab` model should be available in the dropdown in the tool. Make sure to select it.
![WebUI](images/ollama-selectmodel.png)

At the bottom of the screen is the prompt line where you can ask a question of the model. The Milvus example in a prior notebook was exploring climate change and the impact on business.
> In this example we will use Wikipedia articles on a specific topic, Climate Change. We want to explore answering business questions related to this topic. As the environment continues to change, businesses will need to take into consideration how these changes will impact their operations. Combining additional climate change data alongside business specific data would allow companies to prose meaningful questions and consider alternative outcomes when determining effective business strategies.

Copy the following text into the message line at the bottom of the Open WebUI page.
```
What can my company do to help fight climate change?
```

![WEbUI](images/ollama-message.png)

Once you have copied the text into the message, press the arrow at the end of the link to send the message to the LLM. The screen will update to show that the message is being analyzed by the LLM

![WebUI Runnin](images/ollama-running.png)

The answer to the question will take a significant amount of time (3 minutes approximately) since there are no GPUs on the server we are using and limited cpu resources. The answer that is produced is shown below. Note that the answer may change if the Granite model is updated.
![Answer 1](images/ollama-answer-1.png)

Now we are going to ask the same question again, but use RAG (Retreval Augmented Generation)

> Retrieval Augmented Generation is an AI framework for retrieving facts from an external knowledge base to ground large language models (LLMs) on the most accurate, up-to-date information and to give users insight into LLMs' generative process. This can improve the quality of LLM-generated responses by grounding the model on external sources of knowledge to supplement the LLM’s internal representation of information. Implementing RAG in an LLM-based question answering system has two main benefits: It ensures that the model has access to the most current, reliable facts, and that users have access to the model’s sources, ensuring that its claims can be checked for accuracy and ultimately trusted.

Copy the question below and insert into the message line. This text has been generated by the Milvus notebook. The answer will also take a significant amount of time to complete. Patience is required!
```
Climate protests demand that political leaders take action to prevent climate change. They can take the form of public demonstrations, fossil fuel divestment, lawsuits and other activities. Prominent demonstrations include the School Strike for Climate. In this initiative, young people across the globe have been protesting since 2018 by skipping school on Fridays, inspired by Swedish teenager Greta Thunberg. Mass civil disobedience actions by groups like Extinction Rebellion have protested by disrupting roads and public transport. Litigation is increasingly used as a tool to strengthen climate action from public institutions and companies. Activists also initiate lawsuits which target governments and demand that they take ambitious action or enforce existing laws on climate change. Lawsuits against fossil-fuel companies generally seek compensation for loss and damage. Scientists in the 19th century such as Alexander von Humboldt began to foresee the effects of climate change. In the 1820s, Joseph Fourier proposed the greenhouse effect to explain why Earth's temperature was higher than the Sun's energy alone could explain. Earth's atmosphere is transparent to sunlight, so sunlight reaches the surface where it is converted to heat.

If that fails, managed retreat may be needed. There are economic barriers for tackling dangerous heat impact. Avoiding strenuous work or having air conditioning is not possible for everybody. In agriculture, adaptation options include a switch to more sustainable diets, diversification, erosion control, and genetic improvements for increased tolerance to a changing climate. Insurance allows for risk-sharing, but is often difficult to get for people on lower incomes. Education, migration and early warning systems can reduce climate vulnerability. Planting mangroves or encouraging other coastal vegetation can buffer storms. Ecosystems adapt to climate change, a process that can be supported by human intervention. By increasing connectivity between ecosystems, species can migrate to more favourable climate conditions. Species can also be introduced to areas acquiring a favorable climate. Protection and restoration of natural and semi-natural areas helps build resilience, making it easier for ecosystems to adapt. Many of the actions that promote adaptation in ecosystems, also help humans adapt via ecosystem-based adaptation. For instance, restoration of natural fire regimes makes catastrophic fires less likely, and reduces human exposure. Giving rivers more space allows for more water storage in the natural system, reducing flood risk. Restored forest acts as a carbon sink, but planting trees in unsuitable regions can exacerbate climate impacts.

Oxfam found that in 2023 the wealthiest 10%% of people were responsible for 50%% of global emissions, while the bottom 50%% were responsible for just 8%%. Production of emissions is another way to look at responsibility: under that approach, the top 21 fossil fuel companies would owe cumulative climate reparations of $5.4 trillion over the period 2025–2050. To achieve a just transition, people working in the fossil fuel sector would also need other jobs, and their communities would need investments. Nearly all countries in the world are parties to the 1994 United Nations Framework Convention on Climate Change (UNFCCC). The goal of the UNFCCC is to prevent dangerous human interference with the climate system. As stated in the convention, this requires that greenhouse gas concentrations are stabilised in the atmosphere at a level where ecosystems can adapt naturally to climate change, food production is not threatened, and economic development can be sustained. The UNFCCC does not itself restrict emissions but rather provides a framework for protocols that do. Global emissions have risen since the UNFCCC was signed. Its yearly conferences are the stage of global negotiations. The 1997 Kyoto Protocol extended the UNFCCC and included legally binding commitments for most developed countries to limit their emissions.

Please answer a question using this text. If the question is unanswerable, say "unanswerable".

Question: What can my company do to help fight climate change?
```

![Answer 2](images/ollama-answer-2.png)

The answer ends up being very similar to previous one with some added details at the bottom concerning the UNFCCC convention.

In the Open WebAI screen, select the climate action conversation and delete it.

![Delete](images/ollama-delete-conversation.png)

Once you have deleted the message, create a new chat by selecting the Granite model from the dropdown list. Make sure to select it.
![WebUI](images/ollama-selectmodel.png)


Now take the RAG text and place it into the message line and try again.

```
Climate protests demand that political leaders take action to prevent climate change. They can take the form of public demonstrations, fossil fuel divestment, lawsuits and other activities. Prominent demonstrations include the School Strike for Climate. In this initiative, young people across the globe have been protesting since 2018 by skipping school on Fridays, inspired by Swedish teenager Greta Thunberg. Mass civil disobedience actions by groups like Extinction Rebellion have protested by disrupting roads and public transport. Litigation is increasingly used as a tool to strengthen climate action from public institutions and companies. Activists also initiate lawsuits which target governments and demand that they take ambitious action or enforce existing laws on climate change. Lawsuits against fossil-fuel companies generally seek compensation for loss and damage. Scientists in the 19th century such as Alexander von Humboldt began to foresee the effects of climate change. In the 1820s, Joseph Fourier proposed the greenhouse effect to explain why Earth's temperature was higher than the Sun's energy alone could explain. Earth's atmosphere is transparent to sunlight, so sunlight reaches the surface where it is converted to heat.

If that fails, managed retreat may be needed. There are economic barriers for tackling dangerous heat impact. Avoiding strenuous work or having air conditioning is not possible for everybody. In agriculture, adaptation options include a switch to more sustainable diets, diversification, erosion control, and genetic improvements for increased tolerance to a changing climate. Insurance allows for risk-sharing, but is often difficult to get for people on lower incomes. Education, migration and early warning systems can reduce climate vulnerability. Planting mangroves or encouraging other coastal vegetation can buffer storms. Ecosystems adapt to climate change, a process that can be supported by human intervention. By increasing connectivity between ecosystems, species can migrate to more favourable climate conditions. Species can also be introduced to areas acquiring a favorable climate. Protection and restoration of natural and semi-natural areas helps build resilience, making it easier for ecosystems to adapt. Many of the actions that promote adaptation in ecosystems, also help humans adapt via ecosystem-based adaptation. For instance, restoration of natural fire regimes makes catastrophic fires less likely, and reduces human exposure. Giving rivers more space allows for more water storage in the natural system, reducing flood risk. Restored forest acts as a carbon sink, but planting trees in unsuitable regions can exacerbate climate impacts.

Oxfam found that in 2023 the wealthiest 10%% of people were responsible for 50%% of global emissions, while the bottom 50%% were responsible for just 8%%. Production of emissions is another way to look at responsibility: under that approach, the top 21 fossil fuel companies would owe cumulative climate reparations of $5.4 trillion over the period 2025–2050. To achieve a just transition, people working in the fossil fuel sector would also need other jobs, and their communities would need investments. Nearly all countries in the world are parties to the 1994 United Nations Framework Convention on Climate Change (UNFCCC). The goal of the UNFCCC is to prevent dangerous human interference with the climate system. As stated in the convention, this requires that greenhouse gas concentrations are stabilised in the atmosphere at a level where ecosystems can adapt naturally to climate change, food production is not threatened, and economic development can be sustained. The UNFCCC does not itself restrict emissions but rather provides a framework for protocols that do. Global emissions have risen since the UNFCCC was signed. Its yearly conferences are the stage of global negotiations. The 1997 Kyoto Protocol extended the UNFCCC and included legally binding commitments for most developed countries to limit their emissions.

Please answer a question using this text. If the question is unanswerable, say "unanswerable".

Question: What can my company do to help fight climate change?
```

The answer that comes back is summarized and specific to the information that was provided in our question.

![Answer 3](images/ollama-answer-3.png)

You can use this system to test out responses from the Granite model, but don't expect quick answers!

### Stop the Console and Ollama Service

In [ ]:
%system docker stop open-webui

#### Credits: IBM 2024, George Baklarz [baklarz@ca.ibm.com]